In [24]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import requests
import ast
import aiohttp
import asyncio
import time
import os

os.environ["LIBHDFS_OPTS"] = (
    "-Djava.security.krb5.conf=/home/jioapp/aditya/jiomart_cluster/krb5.conf"
)

fs = pa.hdfs.connect(
    host='10.144.96.170',
    port=8020,
    kerb_ticket="/home/jioapp/aditya/jiomart_cluster/krb5cc_154046"
)

/tmp/ipykernel_101148/3248443765.py:15: FutureWarning: pyarrow.hdfs.connect is deprecated as of 2.0.0, please use pyarrow.fs.HadoopFileSystem instead.
  fs = pa.hdfs.connect(


In [82]:
topn = (
pq
.ParquetDataset("/data1/searchengine/analysis/jiomart/deltacatalogue_20240905/run_20250516/top_10000_stepwise_output_top3.csv/part-0000.snappy.parquet",fs)
.read()
.to_pandas()
)

In [83]:
topn.head()

,query,count,vertical,ground_truth_l1l2l3_category,ground_truth_brand,Normalization,Spell Correction,Phrase Matching,Number Conversion,Rewrite Query,l1l3category_en_string_mv,l1l2category_en_string_mv,open_tokens,tagged,boost_scores_numerical_filter_product_size,brand_string_mv,brickprimarycolor_en_string_mv,boost_l1l3category_en_string_mv,boost_brand_string_mv,boost_brickprimarycolor_en_string_mv
0,fortune oil 15 liter,4.0,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"[""['Fortune']"", ""['Fortune']"", ""['Fortune']"", ...","['fortune', 'oil', '15', 'liter']","['fortune', 'oil', '15', 'liter']","['fortune', 'oil', '15', 'liter']","['fortune', 'oil', '15', 'liter']",{'corrected': {'fortune': {'entities': [['key_...,"['Groceries: Personal Care - Beard Oil', 'Groc...",[],['liter'],"[['fortune', 'brand_string_mv'], ['oil', 'cate...",15000,"[['fortune', '1.0']]",<NA>,<NA>,<NA>,<NA>
1,oil 1 litre,4.0,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"['[]', '[]', '[]', '[]']","['oil', '1', 'litre']","['oil', 'one', 'litre']","['oil', 'one', 'litre']","['oil', 'one', 'litre']",{'corrected': {'oil': {'entities': [['l1l2l4_c...,"['Groceries: Personal Care - Beard Oil', 'Groc...",[],"['one', 'litre']","[['oil', 'category'], ['one', 'open'], ['litre...",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
2,refined oil 5l,4.0,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"['[]', '[]', '[]', '[]']","['refined', 'oil', '5l']","['refined', 'oil', '5l']","['refined', 'oil', '5l']","['refined', 'oil', '5l']",{'corrected': {'refined': {'entities': [['key_...,"['Groceries: Personal Care - Beard Oil', 'Groc...",[],"['refined', '5l']","[['refined', 'open'], ['oil', 'category'], ['5...",5000,<NA>,<NA>,<NA>,<NA>,<NA>
3,turmeric,4.0,groceries,['Groceries: Cooking Essentials - Powdered Spi...,"['[]', '[]', '[]', '[]']",['turmeric'],['turmeric'],['turmeric'],['turmeric'],{'corrected': {'turmeric': {'entities': [['dis...,['Groceries: Home Care - Kumkum/Turmeric/Sand'],[],[],"[['turmeric', 'category']]",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
4,dhania,3.0,groceries,['Groceries: Fruits & Vegetables - Leafy Veget...,"['[]', '[]', '[]']",<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [84]:
topn = topn[["query","vertical","ground_truth_l1l2l3_category","ground_truth_brand","l1l3category_en_string_mv","brand_string_mv"]]

In [85]:
topn.head(500)

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv
0,fortune oil 15 liter,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"[""['Fortune']"", ""['Fortune']"", ""['Fortune']"", ...","['Groceries: Personal Care - Beard Oil', 'Groc...","[['fortune', '1.0']]"
1,oil 1 litre,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"['[]', '[]', '[]', '[]']","['Groceries: Personal Care - Beard Oil', 'Groc...",<NA>
2,refined oil 5l,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"['[]', '[]', '[]', '[]']","['Groceries: Personal Care - Beard Oil', 'Groc...",<NA>
3,turmeric,groceries,['Groceries: Cooking Essentials - Powdered Spi...,"['[]', '[]', '[]', '[]']",['Groceries: Home Care - Kumkum/Turmeric/Sand'],<NA>
4,dhania,groceries,['Groceries: Fruits & Vegetables - Leafy Veget...,"['[]', '[]', '[]']",<NA>,<NA>
...,...,...,...,...,...,...
351,wheat flour 5kg,electronics,<NA>,<NA>,[],<NA>
352,wheat flour 5kg,home_lifestyle,<NA>,<NA>,<NA>,<NA>
353,wheat flour 5kg,other_verticals,<NA>,<NA>,<NA>,<NA>
354,white vinegar,beauty,<NA>,<NA>,['Beauty: Fitness - Apple Cider Vinegars'],<NA>


In [86]:
topn['ground_truth_l1l2l3_category'] = topn['ground_truth_l1l2l3_category'].apply(lambda x: [] if pd.isna(x) else x)
topn['ground_truth_brand'] = topn['ground_truth_brand'].apply(lambda x: [] if pd.isna(x) else x)
topn['l1l3category_en_string_mv'] = topn['l1l3category_en_string_mv'].apply(lambda x: [] if pd.isna(x) else x)
topn['brand_string_mv'] = topn['brand_string_mv'].apply(lambda x: [] if pd.isna(x) else x)

In [87]:
topn.head(500)

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv
0,fortune oil 15 liter,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"[""['Fortune']"", ""['Fortune']"", ""['Fortune']"", ...","['Groceries: Personal Care - Beard Oil', 'Groc...","[['fortune', '1.0']]"
1,oil 1 litre,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"['[]', '[]', '[]', '[]']","['Groceries: Personal Care - Beard Oil', 'Groc...",[]
2,refined oil 5l,groceries,['Groceries: Cooking Essentials - Soyabean Oil...,"['[]', '[]', '[]', '[]']","['Groceries: Personal Care - Beard Oil', 'Groc...",[]
3,turmeric,groceries,['Groceries: Cooking Essentials - Powdered Spi...,"['[]', '[]', '[]', '[]']",['Groceries: Home Care - Kumkum/Turmeric/Sand'],[]
4,dhania,groceries,['Groceries: Fruits & Vegetables - Leafy Veget...,"['[]', '[]', '[]']",[],[]
...,...,...,...,...,...,...
351,wheat flour 5kg,electronics,[],[],[],[]
352,wheat flour 5kg,home_lifestyle,[],[],[],[]
353,wheat flour 5kg,other_verticals,[],[],[],[]
354,white vinegar,beauty,[],[],['Beauty: Fitness - Apple Cider Vinegars'],[]


In [88]:
topn['ground_truth_l1l2l3_category'] = topn['ground_truth_l1l2l3_category'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
topn['ground_truth_brand'] = topn['ground_truth_brand'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
topn['l1l3category_en_string_mv'] = topn['l1l3category_en_string_mv'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)
topn['brand_string_mv'] = topn['brand_string_mv'].apply(
    lambda x: ast.literal_eval(x) if isinstance(x, str) else x
)

In [89]:
topn.head(500)

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv
0,fortune oil 15 liter,groceries,"[Groceries: Cooking Essentials - Soyabean Oil,...","[['Fortune'], ['Fortune'], ['Fortune'], ['Fort...","[Groceries: Personal Care - Beard Oil, Groceri...","[[fortune, 1.0]]"
1,oil 1 litre,groceries,"[Groceries: Cooking Essentials - Soyabean Oil,...","[[], [], [], []]","[Groceries: Personal Care - Beard Oil, Groceri...",[]
2,refined oil 5l,groceries,"[Groceries: Cooking Essentials - Soyabean Oil,...","[[], [], [], []]","[Groceries: Personal Care - Beard Oil, Groceri...",[]
3,turmeric,groceries,[Groceries: Cooking Essentials - Powdered Spic...,"[[], [], [], []]",[Groceries: Home Care - Kumkum/Turmeric/Sand],[]
4,dhania,groceries,[Groceries: Fruits & Vegetables - Leafy Vegeta...,"[[], [], []]",[],[]
...,...,...,...,...,...,...
351,wheat flour 5kg,electronics,[],[],[],[]
352,wheat flour 5kg,home_lifestyle,[],[],[],[]
353,wheat flour 5kg,other_verticals,[],[],[],[]
354,white vinegar,beauty,[],[],[Beauty: Fitness - Apple Cider Vinegars],[]


In [92]:
import json
def flatten(x):
    result = []
    for item in x:
        # print(item.strip("[''").strip("'']"))
        if isinstance(item,str):
            result.append(item.strip("[''").strip("'']"))
        elif isinstance(item,list):
            result.extend(item)
    return result
def process_column(x):
    newList = flatten(x)
    newList = list(set(newList))
    return newList

In [93]:
topn["ground_truth_l1l2l3_category"] = topn["ground_truth_l1l2l3_category"].apply(process_column)
topn["ground_truth_brand"] = topn["ground_truth_brand"].apply(process_column)
topn["l1l3category_en_string_mv"] = topn["l1l3category_en_string_mv"].apply(process_column)
topn["brand_string_mv"] = topn["brand_string_mv"].apply(process_column)

In [96]:
topn.to_parquet("cleanedTopN.parquet")

In [97]:
# vertical Level analysis

In [98]:
topn

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv
0,fortune oil 15 liter,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[Fortune],"[Groceries: Cooking Essentials - Coconut Oil, ...","[fortune, 1.0]"
1,oil 1 litre,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[]
2,refined oil 5l,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[]
3,turmeric,groceries,[Groceries: Personal Care - Herbal & Ayurvedic...,[],[Groceries: Home Care - Kumkum/Turmeric/Sand],[]
4,dhania,groceries,[Groceries: Fruits & Vegetables - Leafy Vegeta...,[],[],[]
...,...,...,...,...,...,...
351,wheat flour 5kg,electronics,[],[],[],[]
352,wheat flour 5kg,home_lifestyle,[],[],[],[]
353,wheat flour 5kg,other_verticals,[],[],[],[]
354,white vinegar,beauty,[],[],[Beauty: Fitness - Apple Cider Vinegars],[]


In [99]:
def emptyRows(df):
    
    
    def is_empty(val):
        return val is None or (isinstance(val, list) and len(val) == 0)

    empty_rows = df[
        df[['ground_truth_l1l2l3_category', 'ground_truth_brand', 
            'l1l3category_en_string_mv', 'brand_string_mv']].applymap(is_empty).all(axis=1)
    ]

    empty_queries = empty_rows.copy()

    df_cleaned = df.drop(empty_rows.index).reset_index(drop=True)

    return df_cleaned, empty_queries

In [100]:
cleaned , empty = emptyRows(topn)

/tmp/ipykernel_101148/3748866886.py:8: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['ground_truth_l1l2l3_category', 'ground_truth_brand',


In [103]:
cleaned

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv
0,fortune oil 15 liter,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[Fortune],"[Groceries: Cooking Essentials - Coconut Oil, ...","[fortune, 1.0]"
1,oil 1 litre,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[]
2,refined oil 5l,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[]
3,turmeric,groceries,[Groceries: Personal Care - Herbal & Ayurvedic...,[],[Groceries: Home Care - Kumkum/Turmeric/Sand],[]
4,dhania,groceries,[Groceries: Fruits & Vegetables - Leafy Vegeta...,[],[],[]
...,...,...,...,...,...,...
266,vaseline lip balm,beauty,[],[],[Beauty: Personal Care - Lip Balms],"[1.0, vaseline]"
267,vicco tooth paste,beauty,[],[],[Beauty: Personal Care - Toothpaste],"[1.0, vicco]"
268,water,home_lifestyle,[],[],[Home & Lifestyle: Toys & Games - Sand & Water...,[]
269,white vinegar,beauty,[],[],[Beauty: Fitness - Apple Cider Vinegars],[]


In [109]:
def queryVerticalLevel(df):
    
    def to_list(val):
        return val if isinstance(val, list) else []

    def analyze_row(row):
        gt_cat = set(to_list(row["ground_truth_l1l2l3_category"]))
        pred_cat = set(to_list(row["l1l3category_en_string_mv"]))

        gt_brand = set(to_list(row["ground_truth_brand"]))
        pred_brand = set(to_list(row["brand_string_mv"]))

        return pd.Series({
            "cat_intersection": list(gt_cat & pred_cat),
            "cat_not_detected": list(gt_cat - pred_cat),
            "cat_extra_detected": list(pred_cat - gt_cat),
            "cat_intersection_count":len(gt_cat & pred_cat),
            "cat_miss_count": len(gt_cat - pred_cat),
            "cat_extra_count": len(pred_cat - gt_cat),

            "brand_intersection": list(gt_brand & pred_brand),
            "brand_not_detected": list(gt_brand - pred_brand),
            "brand_extra_detected": list(pred_brand - gt_brand),
            "brand_intersection_count": list(gt_brand & pred_brand),
            "brand_miss_count": len(gt_brand - pred_brand),
            "brand_extra_count": len(pred_brand - gt_brand),
        })

    row_analysis = df.apply(analyze_row, axis=1)

    df_analysis = pd.concat([df, row_analysis], axis=1)
    
    return df_analysis

In [110]:
results = queryVerticalLevel(cleaned)

In [111]:
results

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv,cat_intersection,cat_not_detected,cat_extra_detected,cat_intersection_count,cat_miss_count,cat_extra_count,brand_intersection,brand_not_detected,brand_extra_detected,brand_intersection_count,brand_miss_count,brand_extra_count
0,fortune oil 15 liter,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[Fortune],"[Groceries: Cooking Essentials - Coconut Oil, ...","[fortune, 1.0]",[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",4,0,26,[],[Fortune],"[fortune, 1.0]",[],1,2
1,oil 1 litre,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[],[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",4,0,26,[],[],[],[],1,0
2,refined oil 5l,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[],[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",4,0,26,[],[],[],[],1,0
3,turmeric,groceries,[Groceries: Personal Care - Herbal & Ayurvedic...,[],[Groceries: Home Care - Kumkum/Turmeric/Sand],[],[],[Groceries: Personal Care - Herbal & Ayurvedic...,[Groceries: Home Care - Kumkum/Turmeric/Sand],0,4,1,[],[],[],[],1,0
4,dhania,groceries,[Groceries: Fruits & Vegetables - Leafy Vegeta...,[],[],[],[],[Groceries: Fruits & Vegetables - Leafy Vegeta...,[],0,3,0,[],[],[],[],1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,vaseline lip balm,beauty,[],[],[Beauty: Personal Care - Lip Balms],"[1.0, vaseline]",[],[],[Beauty: Personal Care - Lip Balms],0,0,1,[],[],"[1.0, vaseline]",[],0,2
267,vicco tooth paste,beauty,[],[],[Beauty: Personal Care - Toothpaste],"[1.0, vicco]",[],[],[Beauty: Personal Care - Toothpaste],0,0,1,[],[],"[1.0, vicco]",[],0,2
268,water,home_lifestyle,[],[],[Home & Lifestyle: Toys & Games - Sand & Water...,[],[],[],[Home & Lifestyle: Toys & Games - Sand & Water...,0,0,9,[],[],[],[],0,0
269,white vinegar,beauty,[],[],[Beauty: Fitness - Apple Cider Vinegars],[],[],[],[Beauty: Fitness - Apple Cider Vinegars],0,0,1,[],[],[],[],0,0


In [143]:
import pandas as pd
from collections.abc import Iterable

def analyze_verticals_and_categories(df):
    
    def is_empty(val):
        return val is None or (isinstance(val, Iterable) and len(val) == 0)

    def to_list(val):
        return val if isinstance(val, list) else []
    
    empty_rows = df[
        df[['ground_truth_l1l2l3_category', 'ground_truth_brand', 'l1l3category_en_string_mv', 'brand_string_mv']].applymap(is_empty).all(axis=1)
    ]
    
    empty_queries = empty_rows.copy()
    df_cleaned = df.drop(empty_rows.index).reset_index(drop=True)

    def analyze_verticals(row):
        
        gt_has_values = all(not is_empty(row[col]) for col in ['ground_truth_l1l2l3_category', 'ground_truth_brand'])
        detected_has_values = all(not is_empty(row[col]) for col in ['l1l3category_en_string_mv', 'brand_string_mv'])

        if gt_has_values and detected_has_values:
            return "hit"
        elif gt_has_values and not detected_has_values:
            return "miss"
        elif not gt_has_values and detected_has_values:
            return "extra"
        else:
            return None
        
    def custom_agg(group):
        return pd.Series({
            'hit_count': (group['status'] == 'hit').sum(),
            'extra_count': (group['status'] == 'extra').sum(),
            'miss_count': (group['status'] == 'miss').sum(),
            'hit_verticals': group.loc[group['status'] == 'hit', 'vertical'].unique().tolist(),
            'extra_verticals': group.loc[group['status'] == 'extra', 'vertical'].unique().tolist(),
            'miss_verticals': group.loc[group['status'] == 'miss', 'vertical'].unique().tolist()
        })
    
    df_cleaned['status'] = df_cleaned.apply(analyze_verticals, axis=1)
    query_vertical_comparison = df_cleaned.groupby('query').apply(custom_agg)

    def analyze_row(row):
    
        gt_cat = set(to_list(row["ground_truth_l1l2l3_category"]))
        pred_cat = set(to_list(row["l1l3category_en_string_mv"]))
        
        
        gt_brand = set(to_list(row["ground_truth_brand"]))
        pred_brand = set([str(x).lower() for x in to_list(row["brand_string_mv"]) if isinstance(x, str)])

       
        cat_hit = list(gt_cat & pred_cat)
        cat_miss = list(gt_cat - pred_cat) if gt_cat and not pred_cat else []
        cat_extra = list(pred_cat - gt_cat) if not gt_cat and pred_cat else []
        
        brand_hit = list(gt_brand & pred_brand)
        brand_miss = list(gt_brand - pred_brand) if gt_brand and not pred_brand else []
        brand_extra = list(pred_brand - gt_brand) if not gt_brand and pred_brand else []
        
        return pd.Series({
            "cat_hit": cat_hit,
            "cat_miss": cat_miss,
            "cat_extra": cat_extra,
            "cat_hit_count": len(cat_hit),
            "cat_miss_count": len(cat_miss),
            "cat_extra_count": len(cat_extra),
            
            "brand_hit": brand_hit,
            "brand_miss": brand_miss,
            "brand_extra": brand_extra,
            "brand_hit_count": len(brand_hit),
            "brand_miss_count": len(brand_miss),
            "brand_extra_count": len(brand_extra),
        })

    row_analysis = df_cleaned.apply(analyze_row, axis=1)

    df_analysis = pd.concat([df_cleaned, row_analysis], axis=1)

    return empty_queries, query_vertical_comparison, df_analysis


In [144]:
empty_queries, query_vertical_comparison, df_analysis = analyze_verticals_and_categories(topn)

/tmp/ipykernel_101148/3983317362.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['ground_truth_l1l2l3_category', 'ground_truth_brand', 'l1l3category_en_string_mv', 'brand_string_mv']].applymap(is_empty).all(axis=1)
/tmp/ipykernel_101148/3983317362.py:44: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  query_vertical_comparison = df_cleaned.groupby('query').apply(custom_agg)


In [145]:
query_vertical_comparison

,hit_count,extra_count,miss_count,hit_verticals,extra_verticals,miss_verticals
query,,,,,,
10 kg rice,0,0,1,[],[],[groceries]
50 50 biscuits,1,0,0,[groceries],[],[]
act 2 popcorn,1,1,0,[groceries],[home_lifestyle],[]
adrak,0,0,1,[],[],[groceries]
allout refil,1,0,0,[groceries],[],[]
...,...,...,...,...,...,...
vimbar,0,0,1,[],[],[groceries]
water,0,0,1,[],[],[groceries]
wheat 10kg,0,0,1,[],[],[groceries]


In [146]:
empty_queries

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv
147,10 kg rice,fashion,[],[],[],[]
149,adrak,<NA>,[],[],[],[]
150,almonds 1kg,other_verticals,[],[],[],[]
153,antiseptic liquid,electronics,[],[],[],[]
156,b natural juice,electronics,[],[],[],[]
...,...,...,...,...,...,...
349,wheat 10kg,home_lifestyle,[],[],[],[]
350,wheat 10kg,other_verticals,[],[],[],[]
351,wheat flour 5kg,electronics,[],[],[],[]
352,wheat flour 5kg,home_lifestyle,[],[],[],[]


In [147]:
df_analysis

,query,vertical,ground_truth_l1l2l3_category,ground_truth_brand,l1l3category_en_string_mv,brand_string_mv,status,cat_hit,cat_miss,cat_extra,cat_hit_count,cat_miss_count,cat_extra_count,brand_hit,brand_miss,brand_extra,brand_hit_count,brand_miss_count,brand_extra_count
0,fortune oil 15 liter,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[Fortune],"[Groceries: Cooking Essentials - Coconut Oil, ...","[fortune, 1.0]",hit,[Groceries: Cooking Essentials - Groundnut Oil...,[],[],4,0,0,[],[],[],0,0,0
1,oil 1 litre,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[],miss,[Groceries: Cooking Essentials - Groundnut Oil...,[],[],4,0,0,[],[],[],0,1,0
2,refined oil 5l,groceries,[Groceries: Cooking Essentials - Groundnut Oil...,[],"[Groceries: Cooking Essentials - Coconut Oil, ...",[],miss,[Groceries: Cooking Essentials - Groundnut Oil...,[],[],4,0,0,[],[],[],0,1,0
3,turmeric,groceries,[Groceries: Personal Care - Herbal & Ayurvedic...,[],[Groceries: Home Care - Kumkum/Turmeric/Sand],[],miss,[],[],[],0,0,0,[],[],[],0,1,0
4,dhania,groceries,[Groceries: Fruits & Vegetables - Leafy Vegeta...,[],[],[],miss,[],[Groceries: Fruits & Vegetables - Leafy Vegeta...,[],0,3,0,[],[],[],0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
266,vaseline lip balm,beauty,[],[],[Beauty: Personal Care - Lip Balms],"[1.0, vaseline]",extra,[],[],[Beauty: Personal Care - Lip Balms],0,0,1,[],[],"[1.0, vaseline]",0,0,2
267,vicco tooth paste,beauty,[],[],[Beauty: Personal Care - Toothpaste],"[1.0, vicco]",extra,[],[],[Beauty: Personal Care - Toothpaste],0,0,1,[],[],"[1.0, vicco]",0,0,2
268,water,home_lifestyle,[],[],[Home & Lifestyle: Toys & Games - Sand & Water...,[],None,[],[],[Home & Lifestyle: Toys & Games - Sand & Water...,0,0,9,[],[],[],0,0,0
269,white vinegar,beauty,[],[],[Beauty: Fitness - Apple Cider Vinegars],[],None,[],[],[Beauty: Fitness - Apple Cider Vinegars],0,0,1,[],[],[],0,0,0
